# Part 5 : Recommendation sytem, we consolidate all notebook and define a final prediction

To do this notebook I used all previous notebook.  
It's better to read them before going to this one.

# The summary of the notebook is written below

I. Import useful library and python file containing our functions  




# I. Import useful library and python file containing our functions

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import Function_05 as f5
import pandas as pd
import time
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline, KNNWithZScore
from surprise.model_selection import KFold
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
import matplotlib.pyplot as plt
import seaborn as sns
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
import operator
import statistics
from sklearn.metrics import mean_squared_error 

# set the graphs to show in the jupyter notebook
%matplotlib inline

# II. Recommendation on user details

## A. Retrieving useful dataframe

In [2]:
df_non_stand = pd.read_csv('non_stand_user_perso_info.csv')
Product_mapped = pd.read_csv("product_info_mapped.csv")
user_item_matrix = pd.read_hdf("user_item_matrix.hdf","user_item_matrix")
df_stand = pd.read_hdf('standadize_df_user_personal_info.hdf','standadize_df_user_personal_info')


## B. Doing prediction for a specific user

In [3]:
index = False
cust_id = 266785
threshold = 0.99
max_neighbours = 10
customer_index, customer_id = f5.select_customer(index , cust_id, df_non_stand)
pred_unknown_01, pred_known_01 = f5.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,df_non_stand,Product_mapped,user_item_matrix)

## C. Display result

In [4]:
pred_unknown_01

,object_name,object_id,mean_from_similar_user
14,Books_Academic,5_12,4.500000
4,Footwear_Women,2_3,4.000000
21,Home and kitchen_Bath,6_11,4.000000
10,Electronics_Audio and video,3_10,4.000000
1,Clothing_Women,1_1,3.666667
8,Electronics_Personal Appliances,3_8,3.500000
15,Books_Non-Fiction,5_10,3.000000
22,Home and kitchen_Tools,6_12,2.666667
7,Electronics_Computers,3_5,2.666667
6,Electronics_Mobiles,3_4,2.500000


In [5]:
pred_known_01

,object_name,object_id,mean_from_similar_user,real_rank_from_266785
0,Bags_Mens,4_1,4.000000,3.0
1,Footwear_Mens,2_1,3.600000,4.0
2,Footwear_Kids,2_4,2.666667,3.5
3,Home and kitchen_Kitchen,6_10,2.500000,5.0
4,Books_Children,5_11,1.000000,5.0


## D. Information about this method

The RMSE tends toward 2.00 when we try this method on 100 differents users.

# III. Collaborative filtering memory based

## A. Retrieving useful dataframe

In [6]:
ratings_table = pd.read_csv('rating_table.csv')
ratings_table = ratings_table.rename(columns={"cust_id": "userId", "object_id": "ItemId", "rank": "rating", "object_name": "Item_Name"})
ratings_table = ratings_table.groupby(['userId', 'ItemId', 'Item_Name'])[['rating']].mean().reset_index()
list_user = ratings_table.userId.unique().tolist()

## B. Preparing datesets

In [7]:
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user_id, item_id and ratings (in that order).
data = Dataset.load_from_df(ratings_table[['userId', 'ItemId', 'rating']], reader)
trainsetfull = data.build_full_trainset()

#we create here an anti_set
anti_set = data.build_full_trainset().build_anti_testset()

#we create here a nice_set
nice_set = data.build_full_trainset().build_testset()

## C. Train the algorithm

In [8]:
algo, _, _, _, _ =  f5.compute_result(algor = KNNBasic, my_k = 5, my_min_k = 2, my_sim_option = {'name': 'pearson', 'user_based': False}, data = data)
algo.fit(trainsetfull)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


## D. Display prediction

* On nice_set

In [9]:
predictions_nice = algo.test(nice_set)
all_known = pd.DataFrame(predictions_nice)
all_known.head()

,uid,iid,r_ui,est,details
0,266783,1_4,2.0,2.000000,"{'actual_k': 2, 'was_impossible': False}"
1,266783,2_1,4.0,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."
2,266783,5_10,2.0,2.000000,"{'actual_k': 2, 'was_impossible': False}"
3,266784,3_4,2.0,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."
4,266784,5_10,3.0,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."


* On anti_set

In [10]:
predictions_anti = algo.test(anti_set)
all_unknown = pd.DataFrame(predictions_anti)
all_unknown.head()

,uid,iid,r_ui,est,details
0,266783,3_4,2.999398,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."
1,266783,5_7,2.999398,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."
2,266783,2_4,2.999398,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."
3,266783,4_1,2.999398,3.966358,"{'actual_k': 2, 'was_impossible': False}"
4,266783,5_11,2.999398,2.999398,"{'was_impossible': True, 'reason': 'Not enough..."


* For a specific user

In [11]:
user_id_to_predict = 266784
treshold = 3.0

known = f5.prediction_with_treshold(user_id_to_predict, treshold, all_known)
unknown = f5.prediction_with_treshold(user_id_to_predict, treshold, all_unknown)

known.head()

,uid,iid,r_ui,est,details


In [12]:
unknown.head()

,uid,iid,r_ui,est,details
34,266784,6_2,2.999398,4.965722,"{'actual_k': 2, 'was_impossible': False}"
23,266784,4_1,2.999398,4.572032,"{'actual_k': 2, 'was_impossible': False}"
38,266784,5_3,2.999398,4.525112,"{'actual_k': 2, 'was_impossible': False}"
30,266784,4_4,2.999398,4.498763,"{'actual_k': 2, 'was_impossible': False}"


## E. Information about this method

In [13]:
print('On the nice_set, the RMSE score is :', accuracy.rmse(predictions_nice))

RMSE: 0.5442
On the nice_set, the RMSE score is : 0.5441917914334575


The RMSE score is lower, we have for this method 0.62

In [14]:
reduce_prediction = [val for val in predictions_nice if val[4]['was_impossible'] == False]
print('On the nice_set but only with real prediction, the RMSE score is :', accuracy.rmse(reduce_prediction))

RMSE: 0.2166
On the nice_set but only with real prediction, the RMSE score is : 0.21658931145795912


If we only take into account prediction that are really done meaning more than 2 similar user to compare with, the RMSE score is even lower and reach 0.47

# IV. Collaborative filtering user_based

## A. Retrieving useful dataframe

In [15]:
user_item_matrix = pd.read_hdf("user_item_matrix.hdf","user_item_matrix")
Product_map = pd.read_csv("product_info_mapped.csv")

## B. Selecting a user_id and do prediction

In [16]:
current_user = 266784
#we set here the max value at 5
max_value = 5
similar_user_indices = f5.similar_users(current_user, user_item_matrix, k=6)
pred_known_03 = f5.recommend_item_for_all(current_user, similar_user_indices, max_value, user_item_matrix,Product_map, True)
pred_unknown_03 = f5.recommend_item_for_all(current_user, similar_user_indices, max_value, user_item_matrix,Product_map, False)

## C. Display prediction for a specific user

In [17]:
pred_unknown_03.head()

,object_name,object_id,mean_from_similar_user
22,Home and kitchen_Tools,6_12,3.0
17,Books_Comics,5_3,3.0
14,Books_Academic,5_12,1.0
1,Clothing_Women,1_1,0.0
21,Home and kitchen_Bath,6_11,0.0


In [18]:
pred_known_03.head()

,object_name,object_id,mean_from_similar_user,real_rank_from_266784
0,Books_Fiction,5_7,4.2,5.0
1,Electronics_Mobiles,3_4,4.0,2.0
2,Books_Non-Fiction,5_10,2.8,3.0


## D. Information about this method

The RMSE score tends to 1.07, meaning that the estimated ratings on average are about 1.07 higher or lower than the actual ratings.
Here, our scale is from 0 to 5, so it's is not a bit better than previous notebooks.

# V. Create the consolidated Data frame

## A. Set up user

In [19]:
specific_user = 266794

## B. Result for 01

In [20]:
index = False
cust_id = specific_user
threshold = 0.99
max_neighbours = 10

customer_index, customer_id = f5.select_customer(index , cust_id, df_non_stand)
pred_unknown_01, pred_known_01 = f5.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,df_non_stand,Product_mapped,user_item_matrix)

## C. Result for 02

In [21]:
user_id_to_predict = specific_user
treshold = 0

pred_known_02 = f5.prediction_with_treshold(user_id_to_predict, treshold, all_known)
pred_unknown_02 = f5.prediction_with_treshold(user_id_to_predict, treshold, all_unknown)

## D. Result for 03

In [22]:
current_user = specific_user
max_value = 5

similar_user_indices = f5.similar_users(current_user, user_item_matrix, k=6)
pred_known_03 = f5.recommend_item_for_all(current_user, similar_user_indices, max_value, user_item_matrix,Product_map, True)
pred_unknown_03 = f5.recommend_item_for_all(current_user, similar_user_indices, max_value, user_item_matrix,Product_map, False)

## E. Merging all known values for this user

In [23]:
A,B,C = 1,1,1
consolidated_known = f5.consolidate_know_function(pred_known_01,pred_known_02, pred_known_03, A,B,C)

In [24]:
consolidated_known

,object_name,object_id,real_rank,pred_01,pred_02,pred02_worked,pred_03,total,div_by,final_estimation
0,Bags_Mens,4_1,2.0,5.000000,2.312197,True,0.000000,7.312197,2.0,3.656098
1,Books_Academic,5_12,4.0,3.666667,3.633484,True,3.800000,11.100150,3.0,3.700050
2,Clothing_Kids,1_3,3.5,3.333333,3.456238,True,3.875000,10.664571,3.0,3.554857
3,Footwear_Women,2_3,2.5,3.000000,2.679704,True,4.000000,9.679704,3.0,3.226568
4,Books_Children,5_11,3.0,3.000000,2.779040,True,2.333333,8.112373,3.0,2.704124
5,Electronics_Personal Appliances,3_8,4.0,2.500000,3.779241,True,3.625000,9.904241,3.0,3.301414
6,Electronics_Audio and video,3_10,1.5,0.000000,2.006706,True,2.500000,4.506706,2.0,2.253353
7,Bags_Women,4_4,3.0,0.000000,2.764587,True,2.500000,5.264587,2.0,2.632294


## E. Finding the best way to consolidated all the metrics by running a loop over user

In [26]:
def find_metrics(specific_user):
    
    #finding for 01 :     
    customer_index, customer_id = f5.select_customer(index , specific_user, df_non_stand)
    _, pred_known_01 = f5.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,
                                                      df_non_stand,Product_mapped,user_item_matrix)

    #finding for 02 : 
    pred_known_02 = f5.prediction_with_treshold(specific_user, treshold, all_known)
     
    #finding for 03 : 
    similar_user_indices = f5.similar_users(specific_user, user_item_matrix, k=6)
    pred_known_03 = f5.recommend_item_for_all(specific_user, similar_user_indices, max_value, user_item_matrix,Product_map, True)
       
    consolidated_known = f5.consolidate_know_function(pred_known_01,pred_known_02, pred_known_03, A,B,C)
    mse, rmse = f5.meatrics_mse_rmse_consolidated(consolidated_known)
    MSE.append(mse)
    RMSE.append(rmse)

In [46]:
RMSE = []
MSE = []
A,B,C = 1,3,2

t1 = time.time()

df_result = pd.DataFrame(columns=['A','B','C','mean_mse', 'mean_rmse']) 

for A in [1, 2, 3]:
    for B in [1,2,3,4]:
        for C in [1,2,3,4]:
            for i, usr in enumerate(list_user):
                find_metrics(usr)

                if i == 14:
                    
    
                    L = [[A,B,C,statistics.mean(MSE), statistics.mean(RMSE)]]
                    RMSE = []
                    MSE = []
                    
                    df_result = df_result.append(pd.DataFrame(L,columns=['A','B','C','mean_mse', 'mean_rmse'] ))
                    
                    break
                    
t2 = time.time()

print('took in sec :', (t2 - t1))   

result_ABC = df_result.sort_values(by=['mean_rmse'], ascending = True)
result_ABC.to_csv('result_ABC.csv')
result_ABC.head(20)

took in sec : 705.8702399730682


,A,B,C,mean_mse,mean_rmse
0,1,4,1,0.543412,0.663720
0,1,4,2,0.540224,0.680957
0,1,4,3,0.558825,0.702578
0,1,3,1,0.598596,0.717752
0,1,4,4,0.585029,0.723503
0,1,3,2,0.602743,0.734348
0,1,3,3,0.629365,0.755723
0,1,3,4,0.661253,0.775772
0,1,2,1,0.709443,0.803656
0,2,4,2,0.709443,0.803656


Now that we find a RMSE at 0.663 we can keep this set of value A,B,C at 1,4,1.   
Meaning that we accord much more importance at B so the method 02 and less at method 01 and 03

## F. Merging all unknown values for our specific user

In [52]:
A,B,C = 1,4,1

In [53]:
final_prediction = f5.consolidate_unknown_function(pred_unknown_01,pred_unknown_02, pred_unknown_03, A,B,C).sort_values(by=['final_estimation'], ascending = False)
final_prediction

,object_name,object_id,rank_by_default,pred_01,pred_02,pred02_worked,pred_03,total,div_by,final_estimation
7,Clothing_Women,1_1,2.999398,3.333333,3.955848,True,3.0,22.156727,6.0,3.692788
1,Home and kitchen_Bath,6_11,2.999398,5.000000,3.340977,True,0.0,18.363909,5.0,3.672782
3,Books_DIY,5_6,2.999398,4.666667,3.299175,True,0.0,17.863367,5.0,3.572673
0,Books_Comics,5_3,2.999398,5.000000,2.835749,True,0.0,16.342997,5.0,3.268599
13,Home and kitchen_Kitchen,6_10,2.999398,2.000000,3.668963,True,2.0,18.675851,6.0,3.112642
2,Electronics_Computers,3_5,2.999398,4.666667,2.952015,True,2.0,18.474728,6.0,3.079121
5,Home and kitchen_Furnishing,6_2,2.999398,4.000000,3.325469,True,1.0,18.301877,6.0,3.050313
4,Books_Fiction,5_7,2.999398,4.000000,2.659410,True,0.0,14.637639,5.0,2.927528
8,Clothing_Mens,1_4,2.999398,3.000000,2.865182,True,0.0,14.460729,5.0,2.892146
10,Footwear_Mens,2_1,2.999398,2.500000,2.935917,True,0.0,14.243669,5.0,2.848734


So, we finnaly obtain our prediction for our specific customer.  
In the next cell we can see the 3 best option for himm according to my choice ;)

In [54]:
print(f'The Top 5 best prediction for user_id {specific_user} are : \n')
final_prediction.head(5)

The Top 5 best prediction for user_id 266794 are : 



,object_name,object_id,rank_by_default,pred_01,pred_02,pred02_worked,pred_03,total,div_by,final_estimation
7,Clothing_Women,1_1,2.999398,3.333333,3.955848,True,3.0,22.156727,6.0,3.692788
1,Home and kitchen_Bath,6_11,2.999398,5.000000,3.340977,True,0.0,18.363909,5.0,3.672782
3,Books_DIY,5_6,2.999398,4.666667,3.299175,True,0.0,17.863367,5.0,3.572673
0,Books_Comics,5_3,2.999398,5.000000,2.835749,True,0.0,16.342997,5.0,3.268599
13,Home and kitchen_Kitchen,6_10,2.999398,2.000000,3.668963,True,2.0,18.675851,6.0,3.112642
